Данная программа собирает данные об астероидах, пролетающих мимо Земли в заданные пользователем даты.Основные параметры астероидов отражаются в таблице. Пользователь может выбрать один из астероидов в заданном периоде и посмотреть на его орбиту и его положение на ней сегодня и в один из заданных дней. Так же программа выдает сравнительные графики астероидов в заданном периоде по основным параметрам

Для работы программы требуется selenium, установите его если он не установлен у вас, а так же если вы пользуетесь не браузером Google Chrome, то вам придется указать используемый вами бразуер (в программме show_orbit_diagram), а если пользуетесь Chrome, то вам будет необходимо указать путь к Chromedriver, по необходимости сперва его скачав.

Запускайте ячейки по порядку и следуйте инструкциям

In [ ]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver import Chrome
from selenium.webdriver import Safari
from IPython.display import HTML,clear_output
from PIL import Image
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from selenium.common.exceptions import NoSuchElementException
import random
import numpy.random as rand
import telebot
from telebot import types

In [ ]:
#Микро функция, выделяющая красным опасные астероиды#

def color_yes_red(val):                               
    color = 'red' if val == 'Yes' else 'black'
    return 'color: %s' % color

#Делаем сслыки в таблице кликабельными#

def make_clickable(val):                              
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

#Выделяем зеленым самый быстрый астероид в выборке и синим тот, который приблизится к Земле ближе всего#

def highlight_max(data, color='lawngreen'):
    attr = 'background-color: {}'.format(color)
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)
    
def highlight_min(data, color='blue'):
    attr = 'background-color: {}'.format(color)
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)
    
#Создание технической таблицы с данными с исппользованием api и pandas#

def info_asteroids(start_date,end_date):
    global table
    entrypoint='https://api.nasa.gov/neo/rest/v1/feed?'
    params={'start_date': start_date, 'end_date':end_date,'api_key':'DEMO_KEY','format':'json'}
    r=requests.get(entrypoint,params=params)
    data=r.json()['near_earth_objects']
    super_spisok=[]
    for i in data.keys():
        for j in data[i]:
            spisok_j=[i]
            spisok_j.append(j['name'])
            spisok_j.append(round(float(j['close_approach_data'][0]['relative_velocity']['kilometers_per_second']),4))
            spisok_j.append(str(round(j['estimated_diameter']['kilometers']['estimated_diameter_min'], 4))+' - '+str(round(j['estimated_diameter']['kilometers']['estimated_diameter_max'], 4)))
            spisok_j.append(float(j['close_approach_data'][0]['miss_distance']['kilometers'])/10000000)
            spisok_j.append(j['close_approach_data'][0]['orbiting_body'])
            if j['is_potentially_hazardous_asteroid']==False:
                 spisok_j.append('No')
            else:
                 spisok_j.append('Yes')
            link=j['nasa_jpl_url']
            spisok_j.append('https'+str(link[4:]))
            super_spisok.append(spisok_j)
    pdframe=pd.DataFrame(super_spisok, columns=['Close approach date','Name','Relative velocity (km/sec)','Estimated diameter (km)','Distance of the closest approach (mln km)',"Orbiting body",'Is it hazardous for Earth?','Link']).sort_values(['Close approach date'], ascending=True)
    pdframe.drop_duplicates(subset ="Relative velocity (km/sec)", keep = 'first', inplace = True)
    pdframe=pdframe.reset_index()
    del pdframe['index']
    table=pdframe.reset_index().rename(columns={'index': 'Number'})

#Создание таблицы с некоторыми косметическими изменениями, думаю пойдет за продвинутый пандас#

def info_asteroids_styled(pdframe):
    if pdframe['Is it hazardous for Earth?'].value_counts()['Yes'] > 5:
        print('Смотрите, чтобы на голову ничего не упало :)')   
    elif pdframe['Is it hazardous for Earth?'].value_counts()['Yes'] == 0:
        print('В эти дни можно не ждать ударов огромным астероидом по голове :)')
    else:
        print('В эти дни будет ясно, с низкой вероятностью удара огромным астероидом по голове :) ')
    pdframe1=pdframe.style.format({'Link':make_clickable}).applymap(color_yes_red).apply(highlight_max,subset=['Relative velocity (km/sec)']).apply(highlight_min,subset=['Distance of the closest approach (mln km)'])
    return pdframe1

#Непосредственно программа вызовающая выбор дат и показывающая таблицу с астероидами#

def get_start():
    while True:
        try:
            print('В базе есть данные с 1900 по 2200 год')
            start_date=input('Введите начало периода наблюдений в формате YYYY-MM-DD: ')
            print('Заметьте, что наблюдаемый период не может быть больше недели')
            end_date=input('Введите конец периода наблюдений в формате YYYY-MM-DD: ')
            info_asteroids(start_date,end_date)
            display(info_asteroids_styled(table))
            break
        except KeyError:
            print('Проверьте еще раз, что вы ввели даты в нужном формате и что разница между начальным и конечным днем не больше 7 дней')


#Открываем браузер и нужную нам страницу и считываем с нее много полезной инфы (кнопок). Используем кучу Selenium'а #

#Если вы используете Chrome, то не забудьте дописать путь к Chromedriver при запуске строчки 206#

#Если же вы используете другой браузер, то импортируете его в первой ячейке, а затем в строчке 206 измените Chrome #
#на ваш браузер и допишите путь к драйверу по необходимости#
    
### Делаем красивый вид сверху на орбиты№

def change_view():
    button_open_menu.click()
    time.sleep(0.2)
    browser.find_element_by_id('lookFromDropdown').click()
    time.sleep(0.1)
    browser.find_element_by_id('view-from-above').click()
    time.sleep(0.2)
    browser.find_element_by_id('lookAtDropdown').click()
    time.sleep(0.1)
    browser.find_element_by_link_text('Sun').click()
    button_close_menu.click()

#Ищем оптимальный набор кликов, чтобы не тыкать 20000 раз на стрелочку вперед#

# Тут нам и пригодится математика и немного numpy#
def optimal_clicks():
    today=np.array([int(x) for x in date_today.split('-')])
    approach_date=np.array([int(x) for x in table.loc[number]['Close approach date'].split('-')])
    diff=approach_date-today
    spisok=[]
    diff1=abs(diff)
    diff2=np.array([diff[0] for x in range(1)]+[diff[1] for x in range(3)]+[diff[2] for x in range(4)])
    spisok.append(2*diff1[0])
    spisok.append(diff1[1]//6)
    spisok.append((diff1[1]-6*spisok[1])//3)
    spisok.append((diff1[1]-6*spisok[1]-3*spisok[2]))
    spisok.append(diff1[2]//10)
    spisok.append((diff1[2]-10*spisok[4])//7)
    spisok.append((diff1[2]-10*spisok[4]-7*spisok[5])//3)
    spisok.append(diff1[2]-10*spisok[4]-7*spisok[5]-3*spisok[6])
    spisok=np.array(spisok)
    clicks=np.where(diff2>0,spisok,-spisok)
    return clicks


#Меняем шаг движения орбит#

def change_step(step):
    button_choose_step.click()
    browser.find_element_by_link_text(step).click()
    


#Показываем картинку орбиты в день близкого приближения в период заданный пользовтелем#

def orbit_on_approach_date():
    steps=['6 Months', '6 Months', '3 Months', '1 Month', '10 Days', '1 Week', '3 Days', '1 Day']
    for i in range(8):
        if optimal_clicks()[i] != 0:
            change_step(steps[i])
            browser.find_element_by_id('date_label').click()
            time.sleep(0.1)
            if optimal_clicks()[i]>0:
                for j in range(optimal_clicks()[i]):
                    time.sleep(0.2)
                    browser.find_element_by_id('date_label').click()
                    button_right.click()
                time.sleep(0.1)
            elif optimal_clicks()[i]<0:
                for j in range(-(optimal_clicks()[i])):
                    time.sleep(0.2)
                    browser.find_element_by_id('date_label').click()
                    button_left.click()
                    browser.find_element_by_id('date_label').click()
                time.sleep(0.1)

# Показываем скриншот того, что сейчас на экране#
def screenshot():
    browser.save_screenshot("screenshot.png")
    display(Image.open('screenshot.png'))
    
#Убираем навигационные клавиши, для красивого скриншота#

def remove_controls():
    button_open_menu.click()
    time.sleep(0.1)
    button_open_settings.click()
    time.sleep(0.1)
    btn_controls.click()
    time.sleep(0.1)
    button_close_settings.click()
    time.sleep(0.1)
    button_close_menu.click()
    
#Итоговая программа по показу орбиты выбранного пользователем астероида#

def show_orbit_diagram():
    global number
    while True:
        number=input('Введите номер астероида из таблицы, чтобы увидеть его орбитальную диаграмму сегодня и в одну из выбранных вами дат, когда он будет ближе всего к Земле: ')
        while True:
            try:
                number=int(number)
            except ValueError:
                print('Кажется введенного вами номера нет в таблице или вы ввели не число, попробуйте снова')
                number=input('Введите номер астероида из таблицы, чтобы увидеть его орбитальную диаграмму сегодня и в одну из выбранных вами дат, когда он будет ближе всего к Земле: ')
            if int(number) in range(len(table.index)):
                break
            else:
                print('Кажется введенного вами номера нет в таблице или вы ввели не число, попробуйте снова')
                number=input('Введите номер астероида из таблицы, чтобы увидеть его орбитальную диаграмму сегодня и в одну из выбранных вами дат, когда он будет ближе всего к Земле: ')

        try:
            global browser
            browser = webdriver.Chrome('') #Вставьте свой путь к Chromedriver#

            ref=table.loc[number]['Link']
            browser.set_window_size(1300, 1300 )
            browser.get(ref)
            browser.find_element_by_link_text("[ show orbit diagram ]").click()
            break
        except NoSuchElementException:
            browser.quit()
            print('Ой-ой, кажется NASA удалило информацию об этом астероиде по непонятным причинам, такое бывает крайне редко, значит вы счастливчик :) Попробуйте посмотреть на орбиту другого астероида')
    time.sleep(0.1)
    browser.switch_to.frame(browser.find_element_by_tag_name("iframe"))
            #кнопки
    global date_today
    date_today=browser.find_element_by_id('date_label').text[:10]
    global button_left
    button_left=browser.find_element_by_id('floating-sim-step-backward')
    global button_right
    button_right=browser.find_element_by_id('floating-sim-step-forward')
    global button_stop
    button_stop=browser.find_element_by_id("floating-sim-stop")
    global button_choose_step
    button_choose_step=browser.find_element_by_id('intervalDropdown')
    global button_open_menu
    button_open_menu=browser.find_element_by_xpath("//a[@onclick=\"openNav()\"]")
    global button_close_menu
    button_close_menu=browser.find_element_by_xpath("//a[@onclick=\"closeNav()\"]")
    global button_open_settings
    button_open_settings=browser.find_element_by_id('sidenav-settings-link')
    global button_close_settings
    button_close_settings=browser.find_element_by_id('close-settings-link')
    global button_fullscreen
    button_fullscreen=browser.find_element_by_id('floating-fullscreen')
    global btn_axes
    btn_axes=browser.find_elements_by_xpath("//label[@class=\"btn btn-default btn-xs active toggle-off\"]")[3]
    global btn_controls
    btn_controls=browser.find_elements_by_xpath("//label[@class=\"btn btn-default btn-xs active toggle-off\"]")[5]
    change_view()
    time.sleep(1)
    button_fullscreen.click()
    time.sleep(0.1)
    button_open_menu.click()
    time.sleep(0.1)
    button_open_settings.click()
    time.sleep(0.1)
    btn_controls.click()
    time.sleep(0.1)
    btn_axes.click()
    button_close_settings.click()
    time.sleep(0.1)
    button_close_menu.click()
    time.sleep(0.3)
    print('Today')
    screenshot()
    time.sleep(0.1)
    remove_controls()
    time.sleep(0.1)
    orbit_on_approach_date()
    time.sleep(0.1)
    remove_controls()
    time.sleep(0.3)
    print('on ' + table.loc[number]['Close approach date'] )
    screenshot()
    browser.quit()


    

#Строим графики#

#Для того, чтобы построить график диаметров астероидов возьмем среднее значение промежутка указнного в таблице#

#Иногда встречаются довольно быстрые или большие астероиды, поэтому чтобы график был более информативным,# 
#программа предложит пользователю убрать сильно выделяющиеся значения#

#Здесь используем seaborn и numpy#

def graphs():
    lst_diam=[]
    for x in table['Estimated diameter (km)'].str.split('-'):
        lst_diam.append(np.sum(np.array(x).astype(float))/2)
    table['Diameter']=np.array(lst_diam)
    
    #define table0
    if np.max(np.array(table['Relative velocity (km/sec)']))-np.max(np.delete(np.array(table['Relative velocity (km/sec)']),np.argmax(table['Relative velocity (km/sec)'])))>10:
        print('У одного астероида слишком большая скорость относительно других астероидов, выкинуть его из выборки, чтобы график был информативным? Да/Нет')
        while True:
            n=input()
            if n in {'Да','да'} :
                table0=table[table['Relative velocity (km/sec)'] < table['Relative velocity (km/sec)'].max()]
                break
            elif n in {'Нет','нет'}:
                table0=table
                break
            else:
                table0=table
                print('Кажется вы не дали четкого ответа, напишите Да или Нет')
    else:
        table0=table
        
    #define table2
    if len(table[table['Diameter'] > 2* np.mean(table['Diameter'])])>0:
        print('У нескольких астероидов диаметр сильно превышает среднее значение по выборке, хотите удалить их, чтобы график был более информативным? Да/Нет')
        while True:
            n=input()
            if n in {'Да','да'} :
                table2=table[table['Diameter'] <= 2*np.mean(table['Diameter'])]
                break
            elif n in {'Нет','нет'}:
                table2=table
                break
            else:
                print('Кажется вы не дали четкого ответа, напишите Да или Нет')
    else:
        table2=table
            
            
        
    #define table1
    if np.max(np.array(table['Distance of the closest approach (mln km)']))-np.max(np.delete(np.array(table['Distance of the closest approach (mln km)']),np.argmax(table['Distance of the closest approach (mln km)'])))>3:
        print('У одного астероида слишком большое расстояние ближайщего приближения к земле относительно других астероидов, выкинуть его из выборки, чтобы график был информативным? Да/Нет')
        while True:
            n=input()
            if n in {'Да','да'} :
                table1=table[table['Distance of the closest approach (mln km)'] < table['Distance of the closest approach (mln km)'].max()]
                break
            elif n in {'Нет','нет'}:
                table1=table
                break
            else:
                print('Кажется вы не дали четкого ответа, напишите Да или Нет')
    else:
        table1=table

    graph, plots = plt.subplots(3, 1)
    pal=sns.color_palette("RdPu")
    sns.barplot(data = table0,palette="Blues_d", x='Number', y='Relative velocity (km/sec)',ax = plots[0])
    sns.barplot(data = table1,palette='Reds_d', x='Number', y='Distance of the closest approach (mln km)', ax = plots[1])
    sns.barplot(data = table2,palette='Greens_d', x='Number', y='Diameter', ax = plots[2])
    if len(table.index)<20:
        n=len(table.index)/2
    else:
        n=len(table.index)/4
    graph.set_size_inches(n, 12)
    graph.tight_layout()


Следующие 3 ячейки позволят вам увидеть непосредственно результат работы программы

In [ ]:
#Итог#

get_start()

#опасные астероиды выделены красным, самый быстрый - зеленым, самый близкий к Земле - синим#

In [ ]:
show_orbit_diagram()

#Иногда возможна ошибка из-за лагов кнопок в браузере, но это не моя вина и я постарался расстваить тайминги#
#так чтобы она не возникала практически никогда, но если она возникнет, то попробуйте еще раз#
#Так же NASA предупреждает, что в положении планет и астероидов на данной диаграмме может быть не совсем точным, для очень больших дат#

In [ ]:
graphs()

#Возможно вам предложат построить графики без учета сильно отклоняющихся от среднего наблюдений#
#Советую запустить дважды и посмотреть разницу#

Далее идет часть с телеграм ботом, первая ячейка это повтор функции из основной части программы, но в ней так же надо поменять браузер или вставить путь к chromedriver и запустить

In [ ]:

def show_orbit_diagram1():
    global number
    number=np.random.choice(np.array([x for x in range(len(table.index))]))
    while True:
        try:
            global browser
            browser = webdriver.Chrome('') #Вставьте свой путь к Chromedriver#

            ref=table.loc[number]['Link']
            browser.set_window_size(1300, 1300 )
            browser.get(ref)
            browser.find_element_by_link_text("[ show orbit diagram ]").click()
            break
        except NoSuchElementException:
            print('Ой-ой, кажется NASA удалило информацию об этом астероиде по непонятным причинам, такое бывает крайне редко, значит вы счастливчик :) Попробуйте посмотреть на орбиту другого астероида')
    time.sleep(0.1)
    browser.switch_to.frame(browser.find_element_by_tag_name("iframe"))
            #кнопки
    global date_today
    date_today=browser.find_element_by_id('date_label').text[:10]
    global button_left
    button_left=browser.find_element_by_id('floating-sim-step-backward')
    global button_right
    button_right=browser.find_element_by_id('floating-sim-step-forward')
    global button_stop
    button_stop=browser.find_element_by_id("floating-sim-stop")
    global button_choose_step
    button_choose_step=browser.find_element_by_id('intervalDropdown')
    global button_open_menu
    button_open_menu=browser.find_element_by_xpath("//a[@onclick=\"openNav()\"]")
    global button_close_menu
    button_close_menu=browser.find_element_by_xpath("//a[@onclick=\"closeNav()\"]")
    global button_open_settings
    button_open_settings=browser.find_element_by_id('sidenav-settings-link')
    global button_close_settings
    button_close_settings=browser.find_element_by_id('close-settings-link')
    global button_fullscreen
    button_fullscreen=browser.find_element_by_id('floating-fullscreen')
    global btn_axes
    btn_axes=browser.find_elements_by_xpath("//label[@class=\"btn btn-default btn-xs active toggle-off\"]")[3]
    global btn_controls
    btn_controls=browser.find_elements_by_xpath("//label[@class=\"btn btn-default btn-xs active toggle-off\"]")[5]
    change_view()
    time.sleep(1)
    button_open_menu.click()
    time.sleep(1)
    button_open_settings.click()
    time.sleep(1)
    btn_axes.click()
    button_close_settings.click()
    time.sleep(0.1)
    button_close_menu.click()
    time.sleep(0.3)
    button_fullscreen.click()
    orbit_on_approach_date()
    time.sleep(0.1)
    change_view()
    time.sleep(1)
    remove_controls()
    time.sleep(0.3)
    screenshot()
    browser.quit()

А вот и сам бот, можете попробовать запустить его у себя, но это возможно только если у вас нет никаких проблем с подключением к телеграму, также я буду открывать программу каждый вечер у себя на компьютере и тогда вы сможете найти моего бота по нику @AsteroidsOrbitsBot 

In [ ]:
bot = telebot.TeleBot('729972514:AAHH_5dp6rDeJRKd8ecGEFlMJcSs2Xz2vo8')
@bot.message_handler(content_types=['text'])
def start(message):
    bot.send_message(message.from_user.id, 'Привет. Каждый день около Земли на расстоянии нескольких миллионов километров пролетает до 7 астероидов. Я могу показать тебе информацию об одном из астероидов, который приблизится к Земле в заданную тобой дату. Напиши дату в формате YYYY-MM-DD. У меня есть данные c 1900 до 2200 года' )
    bot.register_next_step_handler(message, check_date)
def check_date(message):
    try:
        global number
        global table
        global date
        date=message.text
        info_asteroids(date,date)
        bot.send_message(message.from_user.id, 'Немного подождите')
        if int(date.split('-')[0])<1900 or int(date.split('-')[0])>2200:
            bot.send_message(message.from_user.id,'Кажется введенная вами дата выходит за доступимый промежуток от 1900 года до 2200. Попробуйте еще раз')
            bot.register_next_step_handler(message, check_date)
        else:
            show_orbit_diagram1()
            bot.send_photo(message.from_user.id, open('screenshot.png','rb'))
            bot.send_message(message.from_user.id, 'Это астероид ' + table.loc[number]['Name']+ '\nПравда его орбита очень красивая?)\n' +str(date)+ ' этот малыш с примерным размером ' + table.loc[number]['Estimated diameter (km)'] +  ' км будет пролетать мимо Земли на расстоянии всего ' +str(table.loc[number]['Distance of the closest approach (mln km)'])+' миллионов километров со скоростью ' + str(table.loc[number]['Relative velocity (km/sec)']) +' км/с\n\nЕсли вы разбираетесь в астрономии и Вам нужны более специфические параметры, то вы найдете их здесь:\n'+table.loc[number]['Link'] )
            bot.send_message(message.from_user.id, 'Хотите еще? Снова напишите дату в формате YYYY-MM-DD.\nИли же просто не пишите боту ничего')
            bot.register_next_step_handler(message, check_date)
    except KeyError:
        bot.send_message(message.from_user.id, 'Кажется вы ввели не дату или же ваша дата не в формате YYYY-MM-DD. Попробуйте еще раз' )
        bot.register_next_step_handler(message, check_date)
bot.polling(none_stop=True, interval=0)  

На всякий случай, я приложу фотографии как работает бот в архиве, на случай если у вас возникнут проблемы с его запуском. Надеюсь вы поверите мне на слово